<a href="https://colab.research.google.com/github/jvdbms/LangGraph/blob/main/AGENTIC_PROJECTS/00_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Install required libraries
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install pytz

# Set environment variables for email credentials
import os
os.environ['SENDER_EMAIL'] = 'jvd.com.pk@gmail.com'
os.environ['SUPPORT_EMAIL'] = 'support@example.com'  # Update with your support email
os.environ['EMAIL_PASSWORD'] = 'Pakistan@2027'  # Replace with the actual email password

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from google.colab import auth
from googleapiclient.discovery import build
import google.auth
import datetime
import pytz
import csv
import re
import uuid

# Authenticate for Google Sheets
auth.authenticate_user()
creds, project = google.auth.default(scopes=[
    'https://www.googleapis.com/auth/spreadsheets',  # Full access to Sheets API
    'https://www.googleapis.com/auth/drive'          # Full access to Google Drive (if needed)
])
service = build('sheets', 'v4', credentials=creds)

# Define Google Sheet details
SPREADSHEET_ID = '1_TegVRumYPbYqfeYN9zXISoWfm1phdnR7K5J7WiCcx8'  # Replace with your Google Sheet ID
RANGE_NAME = 'Tasks!A:G'  # Updated to include new columns

# Pre-populated task data
predefined_tasks = [
    {"Task ID": f"TASK-{str(i+1).zfill(3)}",
     "Task Title": f"Complete Task {i+1}",
     "Description": f"Detailed description of task {i+1} with specific requirements and steps.",
     "Category": "Work" if i % 3 == 0 else "Personal" if i % 3 == 1 else "Learning",
     "Due Date": f"2024-12-{(i % 31) + 1:02d}",
     "Priority": "High" if i % 4 == 0 else "Medium" if i % 4 == 1 else "Low" if i % 4 == 2 else "Critical",
     "Status": "To Do" if i % 3 == 0 else "In Progress" if i % 3 == 1 else "Done",
     "Date Created": f"2024-11-{(i % 30) + 1:02d}",
     "Start Date": f"2024-11-{(i % 31) + 1:02d}",
     "Assignee": f"Person {i % 5 + 1}",
     "Progress (%)": f"{i % 101}%",
     "Comments": f"Specific comments about the progress of task {i+1}, including challenges and updates."}
    for i in range(50)
]

# Function to initialize Google Sheets with predefined tasks
def initialize_sheet_with_tasks():
    values = [[task["Task ID"], task["Task Title"], task["Description"], task["Category"], task["Due Date"],
               task["Priority"], task["Status"], task["Date Created"], task["Start Date"], task["Assignee"],
               task["Progress (%)"], task["Comments"]] for task in predefined_tasks]
    body = {'values': values}
    service.spreadsheets().values().update(
        spreadsheetId=SPREADSHEET_ID,
        range=RANGE_NAME,
        valueInputOption="RAW",
        body=body
    ).execute()
    print("Predefined tasks have been added to the Google Sheet.")

# Function to get current date and time
def get_current_datetime():
    tz = pytz.timezone('Asia/Karachi')  # Replace with your timezone if needed
    return datetime.datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')

# Function to generate a unique task ID
def generate_task_id():
    return str(uuid.uuid4())[:8]

# Function to add a task to Google Sheets
def add_task_to_sheet(title, description, due_date, priority):
    current_datetime = get_current_datetime()
    task_id = generate_task_id()
    values = [[task_id, title, description, due_date, priority, 'To Do', current_datetime]]  # Adding task details with status
    body = {'values': values}
    result = service.spreadsheets().values().append(
        spreadsheetId=SPREADSHEET_ID,
        range=RANGE_NAME,
        valueInputOption="RAW",
        body=body
    ).execute()
    print(f"Task ID: {task_id} added to Google Sheets")
    log_task_to_file(task_id, title, description, due_date, priority, current_datetime)
    return task_id

# Function to send a task creation notification via email
def send_task_email(title, description, due_date, priority, task_id):
    sender_email = os.environ['SENDER_EMAIL']  # Access sender email from environment variable
    support_email = os.environ['SUPPORT_EMAIL']  # Access support email from environment variable
    password = os.environ['EMAIL_PASSWORD']  # Access email password from environment variable

    # Email content
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = support_email
    message["Subject"] = f"New Task Created - ID: {task_id}"

    current_datetime = get_current_datetime()
    body = (f"Task Details:\nTask ID: {task_id}\nTitle: {title}\nDescription: {description}\n"
            f"Due Date: {due_date}\nPriority: {priority}\nDate Created: {current_datetime}\nStatus: To Do")
    message.attach(MIMEText(body, "plain"))

    # Connect to the SMTP server and send email
    try:
        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(sender_email, password)
            server.sendmail(sender_email, support_email, message.as_string())
        print(f"Task ID: {task_id} notification sent via email!")
    except Exception as e:
        print(f"Failed to send email: {e}")

# Function to log tasks to a local CSV file
def log_task_to_file(task_id, title, description, due_date, priority, date_created):
    file_exists = os.path.isfile("tasks_log.csv")
    with open("tasks_log.csv", mode="a", newline="") as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(["Task ID", "Title", "Description", "Due Date", "Priority", "Status", "Date Created"])
        writer.writerow([task_id, title, description, due_date, priority, "To Do", date_created])
    print("Task logged to local file")

# Function to read tasks from Google Sheets
def read_tasks_from_sheet():
    result = service.spreadsheets().values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=RANGE_NAME
    ).execute()
    rows = result.get('values', [])
    if not rows:
        print("No tasks found.")
    else:
        print("\n--- Task Records ---")
        for row in rows:
            print(row)

# Function to update task status in Google Sheets
def update_task_status(task_id, new_status):
    result = service.spreadsheets().values().get(
        spreadsheetId=SPREADSHEET_ID,
        range=RANGE_NAME
    ).execute()
    rows = result.get('values', [])
    if not rows:
        print("No tasks found.")
        return

    # Find the task in the sheet and update its status
    for i, row in enumerate(rows):
        if row[0] == task_id:
            range_to_update = f'Tasks!F{i+1}'  # Column F is for status
            body = {'values': [[new_status]]}
            service.spreadsheets().values().update(
                spreadsheetId=SPREADSHEET_ID,
                range=range_to_update,
                valueInputOption="RAW",
                body=body
            ).execute()
            print(f"Task ID: {task_id} status updated to {new_status}")
            return

    print(f"Task ID: {task_id} not found")

# Collect task details from user
def collect_task():
    print("\n--- Task Form ---")
    while True:
        title = input("Enter task title: ")
        if title.strip():
            break
        print("Title cannot be empty. Please try again.")

    while True:
        description = input("Enter task description: ")
        if description.strip():
            break
        print("Description cannot be empty. Please try again.")

    while True:
        due_date = input("Enter due date (YYYY-MM-DD): ")
        try:
            datetime.datetime.strptime(due_date, '%Y-%m-%d')
            break
        except ValueError:
            print("Invalid date format. Please try again.")

    while True:
        priority = input("Enter task priority (Low, Medium, High): ")
        if priority.strip().lower() in ['low', 'medium', 'high']:
            break
        print("Invalid priority. Please enter Low, Medium, or High.")

    task_id = add_task_to_sheet(title, description, due_date, priority)
    send_task_email(title, description, due_date, priority, task_id)
    print(f"Task created successfully! Task ID: {task_id}")

# Main function to handle user interactions
def main():
    while True:
        print("\n1. Initialize Tasks in Google Sheets\n2. Create a Task\n3. View Tasks\n4. Update Task Status\n5. Exit")
        choice = input("Choose an option: ")
        if choice == "1":
            initialize_sheet_with_tasks()
        elif choice == "2":
            collect_task()
        elif choice == "3":
            read_tasks_from_sheet()
        elif choice == "4":
            task_id = input("Enter Task ID to update: ")
            new_status = input("Enter new status (To Do, In Progress, Done): ")
            if new_status.strip() in ['To Do', 'In Progress', 'Done']:
                update_task_status(task_id, new_status)
            else:
                print("Invalid status. Please enter To Do, In Progress, or Done.")
        elif choice == "5":
            print("Exiting program. Goodbye!")
            break
        else:
            print("Invalid option. Please try again.")

# Run the main function
if __name__ == "__main__":
    main()



1. Initialize Tasks in Google Sheets
2. Create a Task
3. View Tasks
4. Update Task Status
5. Exit
Choose an option: 5
Exiting program. Goodbye!
